In [ ]:
import os

In [ ]:
os.environ

In [ ]:
api_key = os.getenv('OPENAI_API_KEY')
print(api_key)

In [1]:
!wget 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json'

--2024-07-10 11:09:47--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.009s  

2024-07-10 11:09:47 (68.8 MB/s) - ‘documents.json’ saved [658332/658332]



In [ ]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
n = len(documents)

for i in range(n):
    documents[i]['id'] = i 

In [ ]:
documents[3]

In [ ]:
import hashlib

def generate_document_id(doc):
    combined = f"{doc['course']}-{doc['question']}"
    # combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id


In [ ]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [ ]:
documents[3]

In [ ]:
from collections import defaultdict

In [ ]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [ ]:
len(hashes), len(documents)

In [ ]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

In [ ]:
# hashes['593f7569']
hashes['ca3dc12d']

In [ ]:
import json

In [ ]:
!wget 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-vector-search/eval/documents-with-ids.json'

In [ ]:
# with open('documents-with-ids.json', 'wt') as f_out:
#     json.dump(documents, f_out, indent=2)

In [ ]:
!head documents-with-ids.json

In [ ]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [ ]:
!pip install -U openai

In [ ]:
from openai import OpenAI
client = OpenAI()

In [ ]:
doc = documents[2]
prompt = prompt_template.format(**doc)

In [ ]:
response = client.chat.completions.create(
        model='gpt-3.5-turbo-16k',
        messages=[{"role": "user", "content": prompt}]
    )

json_response = response.choices[0].message.content
json_response

In [ ]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-3.5-turbo-16k',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [ ]:
from tqdm.auto import tqdm

In [ ]:
results = {}

In [ ]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

In [ ]:
import pickle

In [ ]:
with open('results.bin', 'wb') as file:
    pickle.dump(results, file)

print("Dane zostały zapisane do pliku 'results.bin'.")

In [ ]:
!wget 'https://github.com/DataTalksClub/llm-zoomcamp/raw/main/03-vector-search/eval/results.bin'

In [ ]:
with open('results.bin', 'rb') as f_in:
    results = pickle.load(f_in)

In [ ]:
json.dumps(json_questions)

In [ ]:
results['1f6520ca']

In [ ]:
results[doc_id] = json.dumps(json_questions)

In [ ]:
parsed_results = {}

for doc_id, json_questions in results.items():
    parsed_results[doc_id] = json.loads(json_questions)

In [ ]:
print(json_questions)

In [ ]:
json_questions = [
r"How can I resolve the Docker error 'invalid mode: \Program Files\Git\var\lib\postgresql\data'?",
"What should I do if I encounter an invalid mode error in Docker on Windows?",
"What is the correct mounting path to use in Docker for PostgreSQL data on Windows?",
"Can you provide an example of a correct Docker mounting path for PostgreSQL data?",
r"How do I correct the mounting path error in Docker for \Program Files\Git\var\lib\postgresql\data'?"
]

In [ ]:
parsed_results

In [ ]:
documents

In [ ]:
doc_index = {d['id']: d for d in documents}

In [ ]:
doc_index

In [ ]:
if 'c02e79ef' in doc_index:
    print("The key exists in doc_index.")
else:
    print("The key does not exist in doc_index.")

In [ ]:
final_results = []

for doc_id, questions in parsed_results.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))


In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])

In [ ]:
df.to_csv('ground-truth-data.csv', index=False)

In [ ]:
!head ground-truth-data.csv